In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('EmojiData.csv', delimiter=',')

In [36]:
df

,Name,icon,"EmojiXpress, mln","Instagram, mln","Twitter, mln"
0,Grinning,![image](https://pictures.s3.yandex.net/resour...,2.2600,1.020,87
1,Beaming,![image](https://pictures.s3.yandex.net/resour...,19.1000,1.690,150
2,ROFL,![image](https://pictures.s3.yandex.net/resour...,25.6000,0.774,0
3,Tears of Joy,![image](https://pictures.s3.yandex.net/resour...,233.0000,7.310,2270
4,Winking,![image](https://pictures.s3.yandex.net/resour...,15.2000,2.360,264
5,Happy,![image](https://pictures.s3.yandex.net/resour...,22.7000,4.260,565
6,Heart Eyes,![image](https://pictures.s3.yandex.net/resour...,64.6000,11.200,834
7,Kissing,![image](https://pictures.s3.yandex.net/resour...,87.5000,5.130,432
8,Thinking,![image](https://pictures.s3.yandex.net/resour...,6.8100,0.636,0
9,Unamused,![image](https://pictures.s3.yandex.net/resour...,6.0000,0.236,478


In [37]:
top_EmojiXpress = df.sort_values('EmojiXpress, mln', ascending=False).head()

In [38]:
top_Instagram = df.sort_values('Instagram, mln', ascending=False).head()

In [39]:
top_Twitter = df.sort_values('Twitter, mln', ascending=False).head()

In [40]:
top_EmojiXpress

,Name,icon,"EmojiXpress, mln","Instagram, mln","Twitter, mln"
3,Tears of Joy,![image](https://pictures.s3.yandex.net/resour...,233.0,7.310,2270
14,Heart,![image](https://pictures.s3.yandex.net/resour...,118.0,26.000,1080
7,Kissing,![image](https://pictures.s3.yandex.net/resour...,87.5,5.130,432
6,Heart Eyes,![image](https://pictures.s3.yandex.net/resour...,64.6,11.200,834
2,ROFL,![image](https://pictures.s3.yandex.net/resour...,25.6,0.774,0


In [41]:
top_Instagram

,Name,icon,"EmojiXpress, mln","Instagram, mln","Twitter, mln"
14,Heart,![image](https://pictures.s3.yandex.net/resour...,118.0,26.00,1080
6,Heart Eyes,![image](https://pictures.s3.yandex.net/resour...,64.6,11.20,834
3,Tears of Joy,![image](https://pictures.s3.yandex.net/resour...,233.0,7.31,2270
13,Two Hearts,![image](https://pictures.s3.yandex.net/resour...,10.0,5.69,445
7,Kissing,![image](https://pictures.s3.yandex.net/resour...,87.5,5.13,432


In [42]:
top_Twitter

,Name,icon,"EmojiXpress, mln","Instagram, mln","Twitter, mln"
3,Tears of Joy,![image](https://pictures.s3.yandex.net/resour...,233.0000,7.310,2270
14,Heart,![image](https://pictures.s3.yandex.net/resour...,118.0000,26.000,1080
19,Recycle,![image](https://pictures.s3.yandex.net/resour...,0.0333,0.056,932
6,Heart Eyes,![image](https://pictures.s3.yandex.net/resour...,64.6000,11.200,834
15,Heart Suit,![image](https://pictures.s3.yandex.net/resour...,3.3100,1.820,697


'''
We can't sort by the data of Twitter alone - an artifact has fallen into the data.
<p>'Recycle' here looks suspicious'</p>
'''

In [43]:
df[['Name', 'EmojiXpress, mln']].sort_values('EmojiXpress, mln', ascending=False)

,Name,"EmojiXpress, mln"
3,Tears of Joy,233.0000
14,Heart,118.0000
7,Kissing,87.5000
6,Heart Eyes,64.6000
2,ROFL,25.6000
11,Loudly Crying,24.7000
16,Thumbs Up,23.1000
5,Happy,22.7000
12,Kiss Mark,21.7000
1,Beaming,19.1000


In [44]:
df[['Name', 'Instagram, mln']].sort_values('Instagram, mln', ascending=False)

,Name,"Instagram, mln"
14,Heart,26.000
6,Heart Eyes,11.200
3,Tears of Joy,7.310
13,Two Hearts,5.690
7,Kissing,5.130
5,Happy,4.260
10,Sunglasses,3.930
16,Thumbs Up,3.750
12,Kiss Mark,2.870
18,Fire,2.490


In [45]:
df[['Name', 'Twitter, mln']].sort_values('Twitter, mln', ascending=False)

,Name,"Twitter, mln"
3,Tears of Joy,2270
14,Heart,1080
19,Recycle,932
6,Heart Eyes,834
15,Heart Suit,697
11,Loudly Crying,654
5,Happy,565
9,Unamused,478
13,Two Hearts,445
7,Kissing,432
